<a href="https://colab.research.google.com/github/moongsil22/ADP_Rscript/blob/master/MusicVAE_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

포자랩스 - 사전과제

reference
* https://velog.io/@yop5292/MusicVAE-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0
* https://colab.research.google.com/github/magenta/magenta-demos/blob/master/colab-notebooks/MusicVAE.ipynb

# MusicVAE 를 이용한 4마디 드럼 미디 생성하기.
### ___eunsil kim___

[MusicVAE]는 음악을 위한 계층적 반복 변형 오토인코더이며 다양한 모드를 제공하는 음악 시퀀스의 잠재공간을 학습한다.

* 사전 분포로 부터의 무작위 샘플링
* 존재하는 시퀀스 간의 보간.
* 잠재 벡터를 통한 존재하는 시퀀스들의 변형
___
### __AE, RNN, LSTM__
AE(오토인코더) : GAN과 비슷한 결과를 만들지만 미세하게 다르다. GAN은 존재하지 않는 완전한 가상의 것을 만들어 내는 반면, 오토인코더는 입력 테이터의 특징을 효율적으로 담아낸 것을 만들어 낸다. 입력층과 출력층을 동일하게 하며, 입력층보다 적은 수의 노드를 가진 은닉층을 중간에 넣어 차원을 줄여준다. 이때 소실된 데이터를 복원하기 위해 학습을 하고, 이 과정을 통해 입력 데이터의 특징을 효율적으로 응축한 새로운 출력이 나온다.

RNN(순환신경망, Recurrent Neural Network) :
여러 데이터가 순서대로 입력되었을 때 앞서 입력받은 데이터를 잠시 기억해 놓는 방법이다. 기억된 데이터의 중요도를 판단하고 별도의 가중치를 주어 다음 데이터로 넘어간다. 모든 입력 값에 이 작업을 순서대로 실행하므로 다음 층으로 넘어가기 전에 같은 층을 맴도는 것으로 보여 RNN이라고 부른다. 

LSTM(Long Short Term Memory) : RNN의 기울기 소실문제의 단점을 보완한 방법으로, 반복되기 직전에 다음 층으로 기억된 값을 넘길지 여부를 관리하는 단계를 하나 더 추가 한 것이다.
___
### _TFrecord_
[TFrecord]는 이진데이터 포맷으로 데이터를 파일에 직렬화하여 저장하여 대규모 데이터를 효율적으로 학습할수 있게 하여, 데이터의 생성 및 가공 비용을 줄일 수 있다.

___
### __Model 구조__

* Bidirectional Encoder : 2계층 양방향 LSTM 네트워크를 사용한다. 2계층은 완전히 연결된 레이어다.LSTM 2048 상태 사이즈를 모든 레이어와 512의 잠재차원을 위해 사용한다. Bidirectional Encoder 의 사용은 입력시퀀스에 대한 긴 구간 컨텍스트의 latent distribution의 매개변수화를 이상적으로 가져다 준다.

* hierarchical RNN Decoder : Simple RNN이 긴 시퀀스에서는 poor한 샘플링과 재구성의 결과 가져다 준다. 출력시퀀스가 생성됨에 따라 잠재상태의 영향을 사라지게 하기 때문에 (기울기 소실) 이런 결과가 발생 한 것이라 믿고, 이를 보완해주는 계층적 RNN을 선택했다.
원칙적으로 자동회귀 RNN Decoder의 사용은 잠재상태를 무시하고 효과적으로 배우는 것으로 부터 "후방붕괴" 문제를 여전히 허용한다. 
이를 해결하기 위해, 출력 하위 시퀀스 내에서만 상태를 전파하도록 허용함으로써 디코더의 최하위 RNN의 유효범위를 감소시켰다. 

___
### __interpolation__

보간은 비슷하게 반복하는 패턴 A, 간헐적인 희소성을 가진 더 높은 레지스터 B, 그리고 새로운 공유 음악키 안에서 synthesizes 한다. 반면에, 베이스라인 데이터 보간은 하모닉 및 리드미컬한 불협화음 두가지 결과를 단지 mix 한다.   



# 환경설정
시퀀스 합성을 위한 패키지 설치


In [1]:
#@title Setup Environment

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
#import tensorflow as tf
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# def play(note_sequence):
#   mm.play_sequence(note_sequence, synth=mm.midi_synth.fluidsynth)  

def interpolate(model, start_seq, end_seq, num_steps, max_length=16*4,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  # print('Start Seq Reconstruction')
  # play(note_sequences[0])
  # print('End Seq Reconstruction')
  # play(note_sequences[-1])
  # print('Mean Sequence')
  # play(note_sequences[num_steps // 2])
  # print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  # play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 3.1 MB 52.2 MB/s 
     |████████████████████████████████| 2.3 MB 69.5 MB/s 
     |████████████████████████████████| 254 kB 32.5 MB/s 
     |████████████████████████████████| 123 kB 43.4 MB/s 
     |████████████████████████████████| 210 kB 61.9 MB/s 
     |████████████████████████████████| 352 kB 54.7 MB/s 
     |██████████

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

Done


In [3]:
#@title 전처리(midi to tfrecord Transfer)
import pathlib #경로를 객체로 처리하기위해
import zipfile #zip파일 압축 해제를 위해
from magenta.scripts.convert_dir_to_note_sequences import convert_directory #전처리

url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
dir = tf.keras.utils.get_file(origin=url, 
                                   fname='/content/data_dir/data.zip', 
                                   extract=True)
data_dir = pathlib.Path(dir)

#zip파일 압축해제
zipfile.ZipFile('/content/data_dir/data.zip').extractall()


#경로지정
data_root= '/content/groove' # 데이터 경로
csv_file = '/content/groove/info.csv' # midi파일의 session, id, bpm 정보를 담고있음.
tfrec_root = '/content/data_dir/music.tfrecord'# tfrecord파일 경로를 지정

#전처리 함수

convert_directory(data_root,tfrec_root,recursive=True) 

3260318/3260318 [==============================] - 0s 0us/step


In [4]:
#@title train_examples_path, rnn size 등 설정(custom configs.py)
import collections
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}


HParams = contrib_training.HParams

# 모델 config configs.py의 groovae config 사용

#=== 모델구조 ===
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(), #BidirectionalLstmEncoder 사용 
                   lstm_models.GrooveLstmDecoder()), #Hierarchical Decoder
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512, #데이터 배치사이즈
            max_seq_len=16 * 4,  # 4마디 길이지정
            z_size=256, #잠재백터 사이즈
            enc_rnn_size=[512], #인코더 순환 사이즈지정
            dec_rnn_size=[256, 256],#디코더 순환사이즈 지정
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,#드롭아웃
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,

        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='/content/data_dir/music.tfrecord', #데이터 경로 설정
)

In [5]:
#@title epoch=100,tf_slim, trainpath 설정 (custom music_vae_train.py)

import tf_slim
#============ License=====================================================
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#=======================================================================

#=== MusicVAE train script===
#=== magenta/models/music_vae/music_vae_train.py ===

# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
#=== 텐서보드 summary 텍스트===

    examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

    hparams_dict = hparams.values()

#=== 하이퍼 파라미터===# Create a markdown table from hparams.
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
#== 데이터로부터 텐서 입력 ===
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,
    sequence_length) = iterator.get_next()
    input_sequence.set_shape(
    [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
    [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
        sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#=== 훈련 체크포인트, 시간 설정===
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

#==== train loop ====
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            #== 옵티마이저 ===
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op = train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)

                       

def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['groovae_4bar']
    train_dir = '/content/train'
    num_steps = 100 #훈련 epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")

In [6]:
#@title 훈련(run)
run(CONFIG_MAP)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/usr/lo

# 4-Bar Drums Model

미리학습된 4마디 드럼 모델을 가져와서 생성한다.

In [7]:
#@title 미리학습된 모델 가져오기-Load PreTrained Models
#@title Load Pretrained Models

# !gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/*_4bar.tar .
# GROOVAE_4BAR = "groovae_4bar.tar"

drums_models = {}
# One-hot encoded.
drums_config = configs.CONFIG_MAP['groovae_4bar']


# 학습모델 생성하기
#drums_models['groovae_4bar'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=GROOVAE_4BAR)
drums_models['groovae_4bar'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path='/content/train')




## 표본(Sample) 만들기

In [8]:
#@title 샘플생성하기-Generate 4 Samples
#@title Generate 4 samples from the prior of one of the models listed above.
drums_sample_model = "groovae_4bar" #@param ["groovae_4bar"]
temperature = 0.4 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
drums_samples = drums_models[drums_sample_model].sample(n=4, length=16*4, temperature=temperature)


In [9]:
#@title MIDI 샘플 다운받기- download samples
for i, ns in enumerate(drums_samples):
  download(ns, '%s_sample_%d.mid' % (drums_sample_model, i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 보간(Interpolations) 생성하기

In [10]:
#@title 보간에 사용할 MIDI 파일 올리기.
input_drums_midi_data = files.upload().values() or input_drums_midi_data

Saving drum_4bar0.mid to drum_4bar0.mid
Saving drum_4bar1.mid to drum_4bar1.mid
Saving drum_4bar2.mid to drum_4bar2.mid
Saving drum_4bar3.mid to drum_4bar3.mid


In [11]:
#@title MEDI파일로 부터 드럼 추출.4마디 드럼 비트를 슬라이딩윈도우를 사용하여 추출한다.

drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]

extracted_beats = []
for ns in drums_input_seqs:
  extracted_beats.extend(drums_config.data_converter.from_tensors(
      drums_config.data_converter.to_tensors(ns)[1]))
  for i in enumerate(extracted_beats):
   print("Beat", i)
#   play(ns)

Beat (0, tempos {
  qpm: 120.0
}
notes {
  pitch: 36
  velocity: 55
  end_time: 0.125
  instrument: 9
  is_drum: true
}
notes {
  pitch: 47
  velocity: 19
  end_time: 0.125
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 23
  start_time: 0.49318181818181817
  end_time: 0.6181818181818182
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 24
  start_time: 1.1181818181818182
  end_time: 1.2431818181818182
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 23
  start_time: 1.7477272727272726
  end_time: 1.8727272727272726
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 24
  start_time: 2.1227272727272726
  end_time: 2.2477272727272726
  instrument: 9
  is_drum: true
}
notes {
  pitch: 36
  velocity: 16
  start_time: 3.125
  end_time: 3.25
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 24
  start_time: 3.1227272727272726
  end_time: 3.2477272727272726
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
 

In [13]:
#@title 이전 셀에서 선택된 것으로, 2비트 사이 보간하기
drums_interp_model = "groovae_4bar" #@param ["groovae_4bar"]
start_beat = 0 #@param {type:"integer"}
end_beat = 3 #@param {type:"integer"}
start_beat = extracted_beats[start_beat]
end_beat = extracted_beats[end_beat]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
num_steps = 4 #@param {type:"integer"}

drums_interp = interpolate(drums_models[drums_interp_model], start_beat, end_beat, num_steps=num_steps, temperature=temperature)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [114]:
#@title 보간된 MIDI 샘플 다운받기- download samples
download(drums_interp, '%s_interp.mid' % drums_interp_model)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>